# Exercice 1 :

###  1)

In [9]:
%%writefile hello_world_mpi.py

from mpi4py import MPI


COMM = MPI.COMM_WORLD
RANK = COMM.Get_rank()
SIZE = COMM.Get_size()

print("Hello world !")
    

In [2]:
!mpirun -n 4 python3  hello_world_mpi.py 

### 2)

In [3]:
%%writefile hello_world_mpi_r_s.py

from mpi4py import MPI


COMM = MPI.COMM_WORLD
RANK = COMM.Get_rank()
SIZE = COMM.Get_size()

def h_w():
    print("Hello world !",RANK,SIZE)
    
h_w()

In [4]:
!mpirun --oversubscribe -n 8 python3  hello_world_mpi.py #to use more than the real cores existing on the machine

### 3)

In [5]:
%%writefile hello_world_mpi_r_s_0.py

from mpi4py import MPI


COMM = MPI.COMM_WORLD
RANK = COMM.Get_rank()
SIZE = COMM.Get_size()

if RANK==0 :
    print("Hello world !",RANK,SIZE)

In [6]:
!mpirun -n 4 python3  hello_world_mpi.py 

# Exercice 2 :


In [7]:
%%writefile neg_value_exit.py

from mpi4py import MPI

# Initialize MPI environment
COMM = MPI.COMM_WORLD

# Get the rank of the current process
RANK= COMM.Get_rank()

while True:
    if RANK == 0:
        # Only the controller process (rank 0) reads the input
        value = int(input("Enter an integer value (or a negative integer to exit):"))
        
    else:
        value = None
        
    # Send the value to all processes
    value = COMM.bcast(value, root=0)
    
    # Exit the program in case of negative input
    if value < 0:
        break

    
    # Print the received value and the rank of the current process
    print(f"Process {RANK} got {value}")
    COMM.Barrier()

In [8]:
!mpirun -n 4 python3  neg_value_exit.py

# Exercice 3 :

### 1)

In [16]:
%%writefile broadcast_by_ring.py

from mpi4py import MPI

# Initialize MPI environment
COMM = MPI.COMM_WORLD

# Get the rank of the current process
RANK= COMM.Get_rank()

# Get the number of the machine cores
SIZE = COMM.Get_size()


while True:
    
    if RANK == 0:
        
        #Only the controller process (rank 0) reads the input
        value = int(input("Enter an integer value (or a negative integer to exit):"))
        
        COMM.send( value, dest= 1)
        
    
    else :
        
        value = COMM.recv( source= RANK-1)
        
        if RANK < SIZE-1 :
            COMM.send( value + RANK, dest= RANK + 1)
        
        
    if value < 0 :
        
        break
        
   
        
    # Print the received value and the rank of the current process
    
    print(f"Process {RANK} got {value}")
    COMM.Barrier()
        


        
        
        
        
        
        